#### BA2A: Implement MotifEnumeration

In [3]:
def generateAllKmers(k):
    base_neucleotides = ["A", "T", "G", "C"]
    last_base = base_neucleotides #copying base neucleotides in last_base
    current_array = [] #declare an emty list
    for i in range(k-1): #for 2 length kmer the loop will run for i=0 and i=1
        for j in base_neucleotides: #A, T, G, C
            for k in last_base: #copy of A, T, G, C
                current_array.append(j+k) 
        last_base = current_array #update last_base
        current_array = [] #empty current array
    return last_base

def MismatchedMotifSet(text, k, d):
    motifset = set()
    all_Kmer = generateAllKmers(k) #generate all possible k mers
    text_Kmer = textKmer(text, k)  #generate all possible kmers from given text
    for i in range(len(all_Kmer)):
        for j in range(len(text_Kmer)):
          if hammingDistance(all_Kmer[i], text_Kmer[j]) <= d: #for each kmer generated from all possible kmers check hamming distance with main text kmers
            motifset.add(all_Kmer[i]) 
    return motifset
    
def hammingDistance(text1, text2):
    count=0;
    for i in range(len(text1)):
        if(text1[i] != text2[i]):
            count+=1
    return count

def textKmer(text, k):
    my_arr = []
    for i in range(len(text)-k+1):
        my_arr.append(text[i:i+k])
    return my_arr

def DNeighbourhood(genomelist, k,d):
  count = {}

  for genome in genomelist:
    motifset = MismatchedMotifSet(genome, k, d)
    for motif in motifset:
      if motif not in count:
        count[motif] = 1
      else:
        count[motif] += 1
  
  motiflist = []
  for key in count:
    if count[key] == len(genomelist):
      motiflist.append(key)
  return sorted(motiflist)

def WriteToFile(result):
  # outF = open("myOutFile.txt", "w")
  for line in result:
    # write line to output file
    print(line, end=" ")

k = 3
d = 1
with open('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Assignment/Chapter 2/rosalind_ba2a.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
length = len(genomelist)
WriteToFile(DNeighbourhood(genomelist, k, d))


ATA ATT GTT TTT 

#### BA2B: Find a Median String

In [9]:
neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] #remainder is the single symbol
    pref_pattern = NumberToPattern(pref_index, k-1) #recursive call
    
    return pref_pattern+symbol

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def HammingDistance(text1, text2):
    count=0;
    for i in range(len(text1)):
        if(text1[i] != text2[i]):
            count+=1
    return count

def DistanceBetweenPatternsandGenome(genomelist, pattern):
  k = len(pattern)
  distance = 0
  for genome in genomelist:
    hammingDistance = 9999
    genome_kmers = generateTextKmer(genome, k)
    for kmer in  genome_kmers:
      if hammingDistance > HammingDistance(pattern, kmer):
        hammingDistance =  HammingDistance(pattern, kmer)
    distance = distance+hammingDistance
  #print(distance)  
  return distance

def MedianString(genomelist, k):
  distance = 9999;
  limit = pow(4,k)
  for i in range(limit-1):
    pattern = NumberToPattern(i,k)
    if distance >= DistanceBetweenPatternsandGenome(genomelist, pattern):
      #print("pattern, Distance")
      #print(pattern, distance)
      distance = DistanceBetweenPatternsandGenome(genomelist, pattern)
      median = pattern
  return median

k = int(input("Enter the length of kmer: "))
genomelist = []
with open('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Assignment/Chapter 2/rosalind_ba2b.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
#print(genomelist)
ans = MedianString(genomelist, k)
print(ans)

Enter the length of kmer: 3
GAC


#### BA2C: Find a Profile-most Probable k-mer in a String

In [13]:
import numpy as np

Genome = "ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT"
k = 5
profile = []
print("Enter profile matrix value rowwise: ")

arr_entry1 = list(map(float, input().strip().split()))
profile =  np.array(arr_entry1).reshape(4,k)


genomeList = list(Genome) # converting genome string into list


## First plan is to generate all possible kmer  
##
##
kMers = []
for j in range(len(Genome)-k+1):
    currentkmar = Genome[j:j+k] # make a substring
    kMers.append(currentkmar)

kMersSet = list(set(kMers))
#print(kMers)

porbableDictionary = {}
for kMer in kMersSet:
    probability = 1
    kMerList = list(kMer)
    for i in range(len(kMerList)):
        if kMer[i] == 'A':
            probability *= profile[0][i]
        elif kMer[i] == 'C':
            probability *= profile[1][i]
        elif kMer[i] == 'G':
            probability *= profile[2][i]
        elif kMer[i] == 'T':
            probability *= profile[3][i]
    porbableDictionary[kMer] = probability
indexOFmaxValue = max(porbableDictionary, key=porbableDictionary.get)
print("The profile most probable",k, "-mer is:", indexOFmaxValue)

Enter profile matrix value rowwise: 
0.2 0.2 0.3 0.2 0.3 0.4 0.3 0.1 0.5 0.1 0.3 0.3 0.5 0.2 0.4 0.1 0.2 0.1 0.1 0.2
The profile most probable 5 -mer is: CCGAG


#### BA2D: Implement GreedyMotifSearch

In [14]:
import numpy as np


def FormBestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def FormCountMatrix(motifs): # count matrix is 4*k AAGG
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    #print(motif)
    for i in range(len(motif)):
      # print("i: ", i)
      # print(motif[i])
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def FormProfile(motifs): # profile matrix 4*k
  # first we need to calculate count matrix
  countMat = FormCountMatrix(motifs)
  #print("countMat: ", countMat)
  
  profileMatrx = np.zeros((4,len(motifs[0])))
  # now just divide count matrix by number of motifs
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = countMat[row][col] / len(motifs)
  #print("profile Matrix: ", profileMatrx)
  return profileMatrx


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  #column wise
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  #sotring the position
        max = profile[row][col]  #finding maximum vale from column
    consensus += NumberToSymbol(loc)  #creating string by concatanating maz count character
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] #remainder is the single symbol
    pref_pattern = numberToPattern(pref_index, k-1) #recursive call
    
    return pref_pattern+symbol

def Score(motifs):
  profile = FormProfile(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  #motif from motifset
    for i in range(len(motif)): #motif's ith charachter
      if cons[i] != motif[i]:
        score+=1
  return score

def GreedyMotifSearch(genomes, k, t):
  bestMotifs = FormBestMotifs(genomes, k,t)
  #print(bestMotifs)
  kmers = generateTextKmer(genomes[0], k)
  #print("list of kmers in 1st: ", kmers)
  for kmer in kmers:
    motifs = []
    motifs.append(kmer)
    #print("kmer appended: ", kmer)
    for idx in range(1, t):
      # first form profile matrix with existing motifs
      profile_matrix = FormProfile(motifs) 
      # calculate probable kmer and append that kmer to construct our motifs set
      motifs.append(ProfileMostProbableKmer(genomes[idx], k, profile_matrix))
    
    # comparing current motif set with best motif set
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
  return bestMotifs

k = 3
t = 5
with open('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Lab Task 3/rosalind_ba2d.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
  #print(genomelist)
ans = GreedyMotifSearch(genomelist, k, t)
for i in ans:
  print(i)

CAG
CAG
CAA
CAA
CAA


#### BA2E: Implement GreedyMotifSearch with Pseudocounts

In [17]:
import numpy as np


def FormBestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers


def FormCountMatrix(motifs): # count matrix is 4*k AAGG
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    #print(motif)
    for i in range(len(motif)):
      # print("i: ", i)
      # print(motif[i])
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def FormProfile(motifs): # profile matrix 4*k
  # first we need to calculate count matrix
  countMat = FormCountMatrix(motifs)
  #print("count Mat: ", countMat)
  pseudoMat = PseudoCountMatrix(countMat)
  profileMatrx = np.zeros((4,len(motifs[0])))
  # now just divide count matrix by number of motifs
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat):
  pseudo = [1*4]*len(countMat[0])
  pseudoMatrix = pseudo+countMat
 #print("pseudo", pseudo)
  return pseudoMatrix


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  #column wise
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  #sotring the position
        max = profile[row][col]  #finding maximum vale from column
    consensus += NumberToSymbol(loc)  #creating string by concatanating maz count character
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] #remainder is the single symbol
    pref_pattern = numberToPattern(pref_index, k-1) #recursive call
    
    return pref_pattern+symbol

def Score(motifs):
  profile = FormProfile(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  #motif from motifset
    for i in range(len(motif)): #motif's ith charachter
      if cons[i] != motif[i]:
        score+=1
  return score

def GreedyMotifSearchWithPseudoCount(genomes, k, t):
  bestMotifs = FormBestMotifs(genomes, k,t)
  #print(bestMotifs)
  kmers = generateTextKmer(genomes[0], k)
  #print("list of kmers in 1st: ", kmers)
  for kmer in kmers:
    motifs = []
    motifs.append(kmer)
    #print("kmer appended: ", kmer)
    for idx in range(1, t):
      # first form profile matrix with existing motifs
      profile_matrix = FormProfile(motifs) 
      # calculate probable kmer and append that kmer to construct our motifs set
      motifs.append(ProfileMostProbableKmer(genomes[idx], k, profile_matrix))
    
    # comparing current motif set with best motif set
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
  return bestMotifs

def WriteToFile(result):
  for line in result:
    print(line)

k = 3
t = 5
with open('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Assignment/Chapter 2/rosalind_ba2e.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
  #print(genomelist)
ans = GreedyMotifSearchWithPseudoCount(genomelist, k, t)
WriteToFile(ans)

TCA
TCA
TCG
TCA
TCG


#### BA2F: Implement RandomizedMotifSearch

In [18]:
import numpy as np
from random import randint

def FormBestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers


def FormCountMatrix(motifs): # count matrix is 4*k AAGG
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    #print(motif)
    for i in range(len(motif)):
      # print("i: ", i)
      # print(motif[i])
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def FormProfile(motifs): # profile matrix 4*k
  # first we need to calculate count matrix
  countMat = FormCountMatrix(motifs)
  #print("count Mat: ", countMat)
  pseudoMat = PseudoCountMatrix(countMat)
  profileMatrx = np.zeros((4,len(motifs[0])))
  # now just divide count matrix by number of motifs
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat):
  pseudo = [1*4]*len(countMat[0])
  pseudoMatrix = pseudo+countMat
 #print("pseudo", pseudo)
  return pseudoMatrix


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  #column wise
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  #sotring the position
        max = profile[row][col]  #finding maximum vale from column
    consensus += NumberToSymbol(loc)  #creating string by concatanating maz count character
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] #remainder is the single symbol
    pref_pattern = numberToPattern(pref_index, k-1) #recursive call
    
    return pref_pattern+symbol

def Score(motifs):
  profile = FormProfile(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  #motif from motifset
    for i in range(len(motif)): #motif's ith charachter
      if cons[i] != motif[i]:
        score+=1
  return score

def getRandom(n):
  return randint(0, n)

def FormRandomlyMotifs(genomes, k):
  motifs = []
  for genome in genomes:
    start = getRandom(len(genome)-k)
    motifs.append(genome[start:start+k])
  return motifs

def generateMotifsSet(profile_matrix, k, genomes):
  probKmers = []
  for genome in genomes:
    probKmers.append(ProfileMostProbableKmer(genome, k, profile_matrix))
  return probKmers

def RandomizedMotifSearch(genomes, k, t):
  motifs = FormRandomlyMotifs(genomes, k)
  bestMotifs = motifs
  while True:
    profile_matrix = FormProfile(motifs)
    motifs = generateMotifsSet(profile_matrix, k, genomes)
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
    else:
      return bestMotifs

def WriteToFile(result):
  for line in result:
    print(line)

# main function
k = 8
t = 5
with open('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Lab Task 4/rosalind_ba2f.txt', 'r') as f:
  genomelist = [line.strip() for line in f.readlines()]
  
best_score = float("Inf")
best_result = []
for i in range(1000):
  ans = RandomizedMotifSearch(genomelist, k, t)
  curr_score = Score(ans)
  if curr_score <= best_score:
    best_score = curr_score
    best_result = ans
WriteToFile(best_result)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


#### BA2G: Implement GibbsSampler

In [19]:
import numpy as np
import random

def FormBestMotifs(genomes, k,t):
  bmotifs = []
  for genome in genomes:
    bmotifs.append(genome[:k])
  return bmotifs

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers


def FormCountMatrix(motifs): # count matrix is 4*k AAGG
  count = np.zeros((4,len(motifs[0])))
  for motif in motifs:
    #print(motif)
    for i in range(len(motif)):
      # print("i: ", i)
      # print(motif[i])
      if motif[i-1] == 'A':
        count[0][i-1] += 1
      elif motif[i-1] == 'C':
        count[1][i-1] += 1
      elif motif[i-1] == 'G':
        count[2][i-1] += 1
      elif motif[i-1] == 'T':
        count[3][i-1] += 1
  return count

def FormProfile(motifs): # profile matrix 4*k
  # first we need to calculate count matrix
  countMat = FormCountMatrix(motifs)
  #print("count Mat: ", countMat)
  pseudoMat = PseudoCountMatrix(countMat)
  profileMatrx = np.zeros((4,len(motifs[0])))
  # now just divide count matrix by number of motifs
  for row in range(4):
    for col in range(k):
      profileMatrx[row][col] = pseudoMat[row][col] / len(motifs)
  return profileMatrx

def PseudoCountMatrix(countMat):
  pseudo = [1*4]*len(countMat[0])
  pseudoMatrix = pseudo+countMat
 #print("pseudo", pseudo)
  return pseudoMatrix


def ProfileMostProbableKmer(genome, k, profile_matrix):
  maxprob = 0
  kmer = genome[0:k]
  for i in range(1,len(genome) - k +1):
    prob = 1
    pattern = genome[i:i+k]
    for j in range(k):
      l = symbolToNumber(pattern[j])
      prob *= profile_matrix[l][j]
    if prob > maxprob:
      maxprob = prob
      kmer = pattern
  return kmer
  
def symbolToNumber(symbol):
	if symbol == "A":
		return 0
	if symbol == "C":
		return 1
	if symbol == "G":
		return 2
	if symbol == "T":
		return 3

def NumberToSymbol(number):
	if number == 0:
		return "A"
	if number == 1:
		return "C"
	if number == 2:
		return "G"
	if number == 3:
		return "T"

def Consensus(profile):
  consensus = ""
  for col in range(len(profile[0])):  #column wise
    max = 0
    loc = 0
    for row in range(4):
      if profile[row][col] > max:
        loc = row  #sotring the position
        max = profile[row][col]  #finding maximum vale from column
    consensus += NumberToSymbol(loc)  #creating string by concatanating maz count character
  return consensus

neucleotide_base = {0:'A', 1:'C', 2:'G', 3:'T'}
def NumberToPattern(index, k):
    if(k == 1): #base case
        return neucleotide_base[index]
    pref_index = int(index/4)
    rem = int(index%4)
    symbol = neucleotide_base[rem] #remainder is the single symbol
    pref_pattern = numberToPattern(pref_index, k-1) #recursive call
    
    return pref_pattern+symbol

def Score(motifs):
  profile = FormProfile(motifs)
  cons = Consensus(profile)
  score = 0
  for motif in motifs:  #motif from motifset
    for i in range(len(motif)): #motif's ith charachter
      if cons[i] != motif[i]:
        score+=1
  return score

def getRandom(n):
  return randint(0, n)

def FormRandomlyMotifs(genomes, k):
  motifs = []
  for genome in genomes:
    start = getRandom(len(genome)-k)
    motifs.append(genome[start:start+k])
  return motifs

def generateMotifsSet(profile_matrix, k, genomes):
  probKmers = []
  for genome in genomes:
    probKmers.append(ProfileMostProbableKmer(genome, k, profile_matrix))
  return probKmers

def ProfileRandomGeneratedKmer(profile_matrix, k, genome):
  kmers = generateTextKmer(genome, k)
  probs = []
  for kmer in kmers:
    probs.append(Probability(profile_matrix, kmer))
  kmer = WeightedRandom(probs, genome,k)
  return kmer

def WeightedRandom(probs, genome, k):
  s = sum(probs)
  r = random.random()
  counter = 0.0
  for i in range(len(probs)):
    counter += probs[i]
    if counter/s >= r:
      return genome[i:i+k]

def Probability(profile_matrix, kmer):
  prob = 1.0
  for i in range(len(kmer)):
    x = symbolToNumber(kmer[i])
    prob = prob*profile_matrix[x][i]
  return prob

def GibbsSampler(genomes, k, t, N):
  motifs = FormRandomlyMotifs(genomes, k)
  bestMotifs = motifs
  for j in range(N):
    i = getRandom(t-1)
    profile_matrix = FormProfile(motifs[0:i] + motifs[i+1:])
    motifi = ProfileRandomGeneratedKmer(profile_matrix, k, genomes[i])
    motifs[i] = motifi
    if Score(motifs) < Score(bestMotifs):
      bestMotifs = motifs
    else:
      return bestMotifs

def WriteToFile(result):
  for line in result:
    print(line)
    
# main function
k = 8        
t = 5        
N = 100      
with open ('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Lab Task 4/rosalind_ba2g.txt', 'r') as f:
    inputLines = [line.strip() for line in f.readlines()]
  #print(genomelist)
best = GibbsSampler(inputLines, k, t, N)
best_score = Score(best)
best_result = []
for i in range(20):
  ans = GibbsSampler(inputLines, k, t, N)
  curr_score = Score(ans)
  if curr_score <= best_score:
    best_score = curr_score
    best_result = ans
WriteToFile(best_result)

TAAACGGC
AAGGTGCC
ACCGAGAC
GTCGGTGA
CACGTGCA


#### BA2H: Implement DistanceBetweenPatternAndStrings

In [23]:
def DistanceBetweenPatternsandGenome(genomelist, pattern):
  k = len(pattern)
  distance = 0
  for genome in genomelist:
    hammingDistance = float("inf")
    genome_kmers = generateTextKmer(genome, k)
    for kmer in  genome_kmers:
      if hammingDistance > HammingDistance(pattern, kmer):
        hammingDistance =  HammingDistance(pattern, kmer)
    distance = distance+hammingDistance
  #print(distance)  
  return distance

def generateTextKmer(text, k):
  kmers = set()
  for i in range(len(text)-k+1):
    kmers.add(text[i:i+k])
  return kmers

def HammingDistance(text1, text2):
    count=0;
    for i in range(len(text1)):
        if(text1[i] != text2[i]):
            count+=1
    return count


#pattern = input()
with open('/content/drive/MyDrive/Bioinformatics Lab/Rosalind Dataset/Assignment/Chapter 2/rosalind_ba2h.txt', 'r') as f:
  pattern = f.readline().strip()
  listWords = f.readline().strip().split(" ")
  #print(listWords)


print(DistanceBetweenPatternsandGenome(listWords, pattern))


5
